# Teletrabajo en Ecuador: ¿Llegó para quedarse?
## Problame a resolver
Comprender cómo la pandemia ha incrementado la tendencia hacia la modalidad de teletrabajo en Ecuador durante y después de la pandemia.
## Objetivos 
- Comprender en qué medida el teletrabajo se convirtió en una modalidad en crecimiento en Ecuador durante la pandemia.
- Analizar cómo el teletrabajo se incrementó con el paso del tiempo y qué sucedió después de la pandemia.
- Identificar en qué ciudades de Ecuador aumentó el teletrabajo y en cuáles no.


In [1]:
# API docs: http://docs.ckan.org/en/2.9/api/
# Ingesta de datos
import requests
import os 
import json

# Ingesta de datos

In [24]:
# Carpeta para la ingesta de datos
folder_path = "../data/data_analytics_01Project/"
os.makedirs(folder_path, exist_ok=True)

Datos abiertos EC contiene grupos de datos, cada grupo de dato contiene `raw data` y data contextual

In [25]:
def requests_data(group_name:str)->dict:
    """ Solicita un conjunto de datos a la API basado
    en el nombre del grupo

    Args:
        group_name (str): Nombre del grupo de datos

    Returns:
        dict: Respuesta de la API
    """
    query = group_name
    try:
        response = requests.get(f"https://datosabiertos.gob.ec/api/3/action/package_search?q={query}")
        response.raise_for_status()  # Raise an exception for any HTTP error status codes
        data = response.json()
        return data

    except requests.exceptions.RequestException as e:
        # Handle connection errors, timeouts, and other errors related to the request itself
        print("An error occurred:", e)
    except requests.exceptions.HTTPError as e:
        # Handle HTTP error status codes
        print("HTTP error occurred:", e)

# Llama a la funcion y lista los posibles grupos de datos
data = requests_data("teletrabajo")
number_of_suggestions = len(data["result"]["results"])
for i in range(0, number_of_suggestions):
    print(json.dumps(data["result"]["results"][i]["name"],indent = 4, sort_keys = True))

"teletrabajo"


In [27]:
def download_dataset_files(dataset_name, folder_path):
    # Query the API
    data = requests_data(dataset_name)

    if data is None:
        return

    # Working with JSON File
    resources = data["result"]["results"][0]["resources"]
    print("Nombre del conjunto:", dataset_name, "\n")

    [download_resource(index, resource, folder_path) for index, resource in enumerate(resources)]

def download_resource(index, resource, folder_path):
    print("\n")
    print('Recurso numero:', index)

    file_name = resource["name"]
    print("Nombre:", file_name)

    # Resource link
    link = resource["url"]
    # Formatted the link
    formatted_link = str(link).replace('"', '')

    # Download the data
    response = requests.get(formatted_link)
    response.raise_for_status()  # Raise an exception if the download fails

    file_path = os.path.join(folder_path, formatted_link[formatted_link.find('download') + 9:])
    with open(file_path, 'wb') as file:
        file.write(response.content)

    print('Datos descargados.')

In [28]:
dataset_name = "teletrabajo"
download_dataset_files(dataset_name,folder_path)

Nombre del conjunto: teletrabajo 



Recurso numero: 0
Nombre: FichaMetodologicaTeletrabajo
Datos descargados.


Recurso numero: 1
Nombre: MDT_Teletrabajo_2023Abril
Datos descargados.


Recurso numero: 2
Nombre: MDT_Teletrabajo_DD_2022Marzo
Datos descargados.


Recurso numero: 3
Nombre: MDT_Teletrabajo_PM_2022Marzo
Datos descargados.


# Descripción de los datos

In [41]:
import pandas as pd 
import pyexcel as pe

In [29]:
%ls "../data/data_analytics_01Project/"

ficha-metodologica-teletrabajo.pdf  mdt_teletrabajo_dd_2022marzo.ods
mdt_teletrabajo_2023abril.csv       mdt_teletrabajo_pm_2022marzo.ods


`ficha-metodologica-teletrabajo.pdf`: Contiene los detalles de la metologia utilizada






`mdt_teletrabajo_dd_2022marzo.ods`: Contiene la desagregacion de la informacion que se encuentra en la base de datos.

In [47]:
df_temp = pe.get_book(file_name="../data/data_analytics_01Project/mdt_teletrabajo_dd_2022marzo.ods")
df_temp

Hoja1:
+------------------------------+-------------------------------------------------------------------------------------------------------------+
| Institución                  | Ministerio del Trabajo                                                                                      |
+------------------------------+-------------------------------------------------------------------------------------------------------------+
| Identificador                | DD-11-2021Agosto                                                                                            |
+------------------------------+-------------------------------------------------------------------------------------------------------------+
| Documento Referencia         | MDT_Teletrabajo_DD_2021Agosto                                                                               |
+------------------------------+-------------------------------------------------------------------------------------------------------

`mdt_teletrabajo_pm_2022marzo.ods`: Contiene las caracteristicas de la base de datos.

In [48]:
df_temp = pe.get_book(file_name="../data/data_analytics_01Project/mdt_teletrabajo_pm_2022marzo.ods")
df_temp

Hoja1:
+-----------------------------+-------------------------------------------------------------------------------------------------------------+
| Institución                 | Ministerio del Trabajo                                                                                      |
+-----------------------------+-------------------------------------------------------------------------------------------------------------+
| Identificador               | PM-11-2021Agosto                                                                                            |
+-----------------------------+-------------------------------------------------------------------------------------------------------------+
| Documento Referencia        | MDT_Teletrabajo_PM_2021Agosto                                                                               |
+-----------------------------+-------------------------------------------------------------------------------------------------------------+


`mdt_teletrabajo_2023abril.cvs` : Numero de contratos bajo la modalidad de teletrabajo

In [50]:
df_2023 = pd.read_csv("../data/data_analytics_01Project/mdt_teletrabajo_2023abril.csv", delimiter = ";")
df_2023

,Motivo Teletrabajo,Provincia Contrato (grupo),Finalizado,Provincia Contrato,Contratos
0,Emergente,PICHINCHA,Finalizado,PICHINCHA,3
1,Emergente,SANTO DOMINGO DE LOS TSÁCHILAS,Finalizado,SANTO DOMINGO DE LOS TSÁCHILAS,1
2,Emergente,PICHINCHA,Finalizado,PICHINCHA,1
3,Habitual,MANABI,Finalizado,MANABI,2
4,Emergente,TUNGURAHUA,Vigente,TUNGURAHUA,1
...,...,...,...,...,...
49212,Habitual,PICHINCHA,Finalizado,PICHINCHA,2
49213,Habitual,LOS RIOS,Finalizado,LOS RIOS,1
49214,Habitual,PICHINCHA,Finalizado,PICHINCHA,1
49215,Habitual,PICHINCHA,Finalizado,PICHINCHA,3


In [51]:
df_2023.columns

Index(['Motivo Teletrabajo', 'Provincia Contrato (grupo)', 'Finalizado',
       'Provincia Contrato', 'Contratos'],
      dtype='object')

In [54]:
url = "https://raw.githubusercontent.com/matardy/datos-abiertos-ec-API/master/data/teletrabajo/mdt_teletrabajo_2022mayo.csv"
df_until_2022 = pd.read_csv(url, delimiter = ";")
df_until_2022

,motivo_teletrabajo,genero,provincia_contrato,provincia_contrato (grupo),finalizado,contratos,anio_fecha_inicio_teletrabajo,anio_fecha_inicio_teletrabajo (grupo),canton_contrato,estado_teletrabajo,fecha_consulta,mes_fecha_inicio_teletrabajo,modo_ejecucion,Número de registros,ruc_empresa
0,Emergente,Femenino,PICHINCHA,PICHINCHA,Vigente,1,2017,Otro,QUITO,Activo,1/6/2022 04:49:26,1.0,Teletrabajo,1,1792040825001
1,Emergente,Femenino,PICHINCHA,PICHINCHA,Vigente,1,2018,Otro,QUITO,Activo,1/6/2022 04:49:26,4.0,Teletrabajo,1,1792692091001
2,Emergente,Femenino,PICHINCHA,PICHINCHA,Vigente,1,2018,Otro,QUITO,Activo,1/6/2022 04:49:26,11.0,Teletrabajo,1,1791109384001
3,Emergente,Femenino,GUAYAS,GUAYAS,Vigente,4,2019,Otro,GUAYAQUIL,Activo,1/6/2022 04:49:26,2.0,Teletrabajo,1,1791256115001
4,Emergente,Femenino,PICHINCHA,PICHINCHA,Vigente,24,2019,Otro,QUITO,Activo,1/6/2022 04:49:26,2.0,Teletrabajo,1,1791256115001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46059,Habitual,Masculino,PICHINCHA,PICHINCHA,Finalizado,1,2022,Otro,QUITO,Activo,1/6/2022 04:49:26,4.0,Teletrabajo,1,1792825482001
46060,Habitual,Masculino,PICHINCHA,PICHINCHA,Finalizado,1,2022,Otro,QUITO,Activo,1/6/2022 04:49:26,4.0,Teletrabajo,1,1792983126001
46061,Habitual,Masculino,SANTO DOMINGO DE LOS TSÁCHILAS,SANTO DOMINGO DE LOS TSÁCHILAS,Finalizado,1,2022,Otro,SANTO DOMINGO DE LOS TSÁCHILAS,Activo,1/6/2022 04:49:26,4.0,Teletrabajo,1,1715337596001
46062,Habitual,Masculino,PICHINCHA,PICHINCHA,Finalizado,2,2022,Otro,QUITO,Activo,1/6/2022 04:49:26,5.0,Teletrabajo,1,1792758726001


In [55]:
df_until_2022

,motivo_teletrabajo,genero,provincia_contrato,provincia_contrato (grupo),finalizado,contratos,anio_fecha_inicio_teletrabajo,anio_fecha_inicio_teletrabajo (grupo),canton_contrato,estado_teletrabajo,fecha_consulta,mes_fecha_inicio_teletrabajo,modo_ejecucion,Número de registros,ruc_empresa
0,Emergente,Femenino,PICHINCHA,PICHINCHA,Vigente,1,2017,Otro,QUITO,Activo,1/6/2022 04:49:26,1.0,Teletrabajo,1,1792040825001
1,Emergente,Femenino,PICHINCHA,PICHINCHA,Vigente,1,2018,Otro,QUITO,Activo,1/6/2022 04:49:26,4.0,Teletrabajo,1,1792692091001
2,Emergente,Femenino,PICHINCHA,PICHINCHA,Vigente,1,2018,Otro,QUITO,Activo,1/6/2022 04:49:26,11.0,Teletrabajo,1,1791109384001
3,Emergente,Femenino,GUAYAS,GUAYAS,Vigente,4,2019,Otro,GUAYAQUIL,Activo,1/6/2022 04:49:26,2.0,Teletrabajo,1,1791256115001
4,Emergente,Femenino,PICHINCHA,PICHINCHA,Vigente,24,2019,Otro,QUITO,Activo,1/6/2022 04:49:26,2.0,Teletrabajo,1,1791256115001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46059,Habitual,Masculino,PICHINCHA,PICHINCHA,Finalizado,1,2022,Otro,QUITO,Activo,1/6/2022 04:49:26,4.0,Teletrabajo,1,1792825482001
46060,Habitual,Masculino,PICHINCHA,PICHINCHA,Finalizado,1,2022,Otro,QUITO,Activo,1/6/2022 04:49:26,4.0,Teletrabajo,1,1792983126001
46061,Habitual,Masculino,SANTO DOMINGO DE LOS TSÁCHILAS,SANTO DOMINGO DE LOS TSÁCHILAS,Finalizado,1,2022,Otro,SANTO DOMINGO DE LOS TSÁCHILAS,Activo,1/6/2022 04:49:26,4.0,Teletrabajo,1,1715337596001
46062,Habitual,Masculino,PICHINCHA,PICHINCHA,Finalizado,2,2022,Otro,QUITO,Activo,1/6/2022 04:49:26,5.0,Teletrabajo,1,1792758726001
